# Digikala data for Apple, Samsung, Xiaomi

In [ ]:
! pip install selenium
! pip install pandas==1.5.3
! pip install unidecode

In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from unidecode import unidecode
import time

### opening digikala

In [3]:
driver = webdriver.Chrome()

In [4]:
url = "https://www.digikala.com/search/category-mobile-phone/"
apple_url = url + "apple/"
xiaomi_url = url + "xiaomi/"
samsung_url = url + "samsung/"

In [5]:
driver.maximize_window()
driver.get(apple_url)

In [6]:
# choose most selled
driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[1]/div/div/div/div[2]/span[3]""").click()

getting links of top 200 latest cards in digikala

In [7]:
apple_links = []
# link_count = how many data you want
link_count = 200
y = 500

for _ in range(link_count // 4):
    cards = driver.find_elements(By.XPATH, f"/html/body/div[1]/div[1]/div[3]/div[3]/div[3]/section[1]/div[2]/div/a")
    for card in cards:
        if (link := card.get_attribute("href")) not in apple_links:
            apple_links.append(link)

    driver.execute_script(f"window.scrollTo(0,{y})")
    y += 500
    time.sleep(1)


In [8]:
len(apple_links)

186

getting data from each link

In [9]:
apple_df = pd.DataFrame(columns=["brand_name", 'model', 'off_price',"price", "storage", "ram", "link"])
apple_df

,brand_name,model,off_price,price,storage,ram,link


In [36]:
# for link in apple_links:
#     driver.get(link)
#     time.sleep(1)
driver.get(apple_links[-1])
time.sleep(2)

wait = WebDriverWait(driver, 10)
# getting the title
title = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, """h1.text-h4"""))).text

# color
try:
    # selecting first color
    driver.find_element(By.CSS_SELECTOR, """div.styles_InfoSectionVariationColor__pX_3M:nth-child(1)""").click()
except:
    pass

# price
try:
    # without off
    price = driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[4]/div[1]/div[8]/div/div/div[2]/div/div[1]/span""").text
    price = unidecode(price.replace(",", ""))
    off_price = "NAN"
except NoSuchElementException:
    try:
        # Na mojud
        driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[3]/div/div/div[1]/p""")
        price = "NAN"
        off_price = "NAN"
    except NoSuchElementException:
        # with off
        price = driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[4]/div[1]/div[8]/div/div/div[1]/div[2]/div[1]/span""").text
        price = unidecode(price.replace(",", ""))
        off_price = driver.find_element(By.XPATH, """/html/body/div[1]/div[1]/div[3]/div[3]/div[2]/div[2]/div[2]/div[2]/div[4]/div[1]/div[8]/div/div/div[1]/div[2]/div[2]/span""").text
        off_price = unidecode(off_price.replace(",", ""))

driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(2)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
driver.execute_script("window.scrollTo(0,0)")
time.sleep(2)
# try:
# driver.find_element(By.CSS_SELECTOR, """ul.d-flex > div:nth-child(3)""")
more = driver.find_element(By.XPATH, """//*[@id="__next"]/div[1]/div[3]/div[3]/div[2]/div[9]/div/section/span""")
driver.execute_script("arguments[0].scrollIntoView();", more)
driver.execute_script("arguments[0].style.border='3px solid red';", more)
more.click()
# except ElementNotInteractableException:
#     pass
# except ElementClickInterceptedException:
#     pass



ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (1459, 2966)
  (Session info: chrome=115.0.5790.173)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF641B84A62+57106]
	(No symbol) [0x00007FF641AFCF52]
	(No symbol) [0x00007FF6419CE2CB]
	(No symbol) [0x00007FF641A0D290]
	(No symbol) [0x00007FF641A0B90D]
	(No symbol) [0x00007FF641A098F5]
	(No symbol) [0x00007FF641A08B05]
	(No symbol) [0x00007FF6419FEE2F]
	(No symbol) [0x00007FF641A269BA]
	(No symbol) [0x00007FF6419FE746]
	(No symbol) [0x00007FF641A26BD0]
	(No symbol) [0x00007FF641A3E522]
	(No symbol) [0x00007FF641A26793]
	(No symbol) [0x00007FF6419FCE81]
	(No symbol) [0x00007FF6419FE064]
	GetHandleVerifier [0x00007FF641E34222+2873042]
	GetHandleVerifier [0x00007FF641E86590+3209792]
	GetHandleVerifier [0x00007FF641E7F3AF+3180639]
	GetHandleVerifier [0x00007FF641C15F25+652245]
	(No symbol) [0x00007FF641B08618]
	(No symbol) [0x00007FF641B047C4]
	(No symbol) [0x00007FF641B048BC]
	(No 

In [102]:
driver.find_element(By.CSS_SELECTOR, """section.mt-4-lg:nth-child(3) > span:nth-child(3)""").click()